In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.1)

def initialize_retriever(file_path):
    cache_dir = LocalFileStore("./.cache/")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(separator="\n", chunk_size=1000, chunk_overlap=100)

    docs = UnstructuredFileLoader(file_path).load_and_split(text_splitter=splitter)
    embeddings = CacheBackedEmbeddings.from_bytes_store(OpenAIEmbeddings(), cache_dir)
    return FAISS.from_documents(docs, embeddings).as_retriever()

retriever = initialize_retriever("./files/1984_chapter_three.txt")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

def load_memory(_):
    return memory.load_memory_variables({}).get("chat_history", [])

final_prompt = ChatPromptTemplate.from_messages([
    ("system", 
        """
        Given the following extracted parts of a long documents and a question, create a final answer.
        If you don't know the answer, just say that you don't know. Don't try to make up an answer.
        -------
        {context}
        """
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

chain = {"chat_history": RunnableLambda(load_memory), "context": retriever, "question": RunnablePassthrough()} | final_prompt | llm 

def invoke_chain(question):
    response = chain.invoke(question)
    memory.save_context(
        inputs={"input":question}, 
        outputs={"output":response.content}
    )
    return response.content

In [2]:
invoke_chain("Was Aaronson guilty of a crime?")

'Yes, according to the extracted text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'

In [3]:
invoke_chain("What message did he write in the table?")

'He wrote "2+2=5" on the table.'

In [4]:
invoke_chain("Who is Julia?")

'Julia is a character mentioned in the text. She is someone who Winston loves and cares for deeply.'

In [5]:
invoke_chain("What was my first question and your answer?")

'Your first question was "Was Aaronson guilty of a crime?" and my answer was "Yes, according to the extracted text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with."'